## Here we train single-hidden-layer convnets with different numbers of nodes

In [110]:
# Import functions
import setGPU1
from io_functions import *
from draw_functions import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
# 1 is signal; 0 is background
train_data, test_data, train_labels, test_labels = train_test(shape=(20, 20, 25), split=0.33)

In [28]:
test_data.shape

(26714, 20, 20, 25)

In [115]:
# Choose our list of number of nodes
num_nodes = [10, 100, 1000, 10000]

for num in num_nodes:
    # Convolutional Layers
    model = Sequential()
    model.add(Convolution2D(10, 4, 4, input_shape = (25, 20, 20), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())

    # Dense layer
    model.add(Dense(num, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, init='uniform', activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='sgd')
    #model.summary()

    # Checkpointing and Early Stopping
    check = ModelCheckpoint(filepath='./tmp/conv2D%s.hdf5'%num, verbose=1)
    early = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

    # Train the network
    my_fit = model.fit(np.swapaxes(train_data, 1, 3), train_labels, nb_epoch=10000, validation_split=0.2,
                       batch_size=1000, verbose=1, callbacks=[check, early])

    # Get predictions
    predicted = model.predict(np.swapaxes(test_data, 1, 3))

    # Store the model, the weights, the loss history, and the predicted and truth labels
    store_model(model, my_fit.history, 'conv2D'+num, (predicted, test_labels))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_21 (Convolution2D)   (None, 10, 17, 17)  4010        convolution2d_input_18[0][0]     
____________________________________________________________________________________________________
maxpooling2d_16 (MaxPooling2D)     (None, 10, 8, 8)    0           convolution2d_21[0][0]           
____________________________________________________________________________________________________
flatten_15 (Flatten)               (None, 640)         0           maxpooling2d_16[0][0]            
____________________________________________________________________________________________________
dense_29 (Dense)                   (None, 10)          6410        flatten_15[0][0]                 
___________________________________________________________________________________________

In [134]:
print('hiya%sdd'%lab)

hiyammdd


In [34]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
GPU run command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10_cnn.py
It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
Note: the data was pickled with Python 2, and some encoding issues might prevent you
from loading it in Python 3. You might have to load it in Python 2,
save it in a different format, load it in Python 3 and repickle it.
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

batch_size = 32
nb_classes = 10
nb_epoch = 200
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

170500096/170498071 [==============================] - 58s    
Untaring file...
X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/200
50000/50000 [==============================] - 41s - loss: 1.7254 - acc: 0.3543 - val_loss: 1.3186 - val_acc: 0.5168
Epoch 2/200
49568/50000 [============================>.] - ETA: 0s - loss: 1.3829 - acc: 0.4996

KeyboardInterrupt: 